# Homework 2
In this assignment, we will use the table operations that we saw in sections 2.2 and 2.4 of the workbook to investigate the `cities` dataset, which contains 251 cities in the western United States and their populations. Throughout this assignment, as in section 2.4 of the workbook, when we refer to a "function", we mean a data frame with two columns, representing the domain and the range.

We will use this table to demonstrate several properties of functions.

**NOTE: Throughout this assignment, when we ask you to "define" or "find" a function, we mean output a table with two columns representing the domain and the range of the function, as in section 2.4.**

In [3]:
from datasets import western_cities as cities
cities.columns = ['City', 'Population', 'State']
cities.head()

,City,Population,State
0,Corona,152374,California
1,Longmont,86270,Colorado
2,Palmdale,152750,California
3,Star Valley Ranch,1503,Wyoming
4,Pomona,149058,California


## Problem 1: Binary relations versus functions

### Part A 
Let $A =$ `State`, $B =$ `City`. Output a table representing the relation $R\subseteq A\times B$ where $\langle a, b \rangle \in R$ if $b$ is in state $a$.

In [4]:
cities[['State', 'City']]

,State,City
0,California,Corona
1,Colorado,Longmont
2,California,Palmdale
3,Wyoming,Star Valley Ranch
4,California,Pomona
...,...,...
246,California,Elk Grove
247,Hawaii,Aiea
248,Colorado,Littleton
249,Colorado,Thornton


Notice that this relation does not define a function from $A$ to $B$, since there are multiple cities within each state. 

### Part B 
Using group by operations, define two _different_ functions from $A =$ `State`, $B =$ `City`. Note that there will be multiple ways to do this.

In [8]:
state_to_city1 = cities.groupby("State", as_index=False)[["State", "City"]].min("Population") #first alphabetically
state_to_city2 = cities.groupby("State", as_index=False)[["State", "City"]].max("Population") #last alphabetically

,State,City
0,Alaska,Wrangell
1,California,West Covina
2,Colorado,Windsor
3,Hawaii,Waipio
4,Washington,Yakima
5,Wyoming,Wright


## Problem 2: Images, preimages and restrictions

### Part A 
Define a function mapping each city to its population, and call this table `city_to_population`. Then compute the preimage of the set $\{x\in \mathbb{N} : x\leq 50,000\}$ under this function. What does this set represent?

In [10]:
city_to_population = cities[['City', 'Population']]
city_to_population

,City,Population
0,Corona,152374
1,Longmont,86270
2,Palmdale,152750
3,Star Valley Ranch,1503
4,Pomona,149058
...,...,...
246,Elk Grove,153015
247,Aiea,9338
248,Littleton,41737
249,Thornton,118772


### Part B
Let $S$ be the set of cities in California. Find the image of this set under the function `city_to_population`.

In [14]:
cities_in_california = cities[cities['State'] == 'California'][['City']]
cities_in_california

,City
0,Corona
2,Palmdale
4,Pomona
6,Santa Rosa
14,Hayward
...,...
232,Concord
234,Salinas
236,Los Angeles
243,Simi Valley


### Part C
Find the restriction of the function `city_to_population` to the set $S$ of cities in California.

In [15]:
city_to_population[city_to_population['City'].isin(cities_in_california['City'])]

,City,Population
0,Corona,152374
2,Palmdale,152750
4,Pomona,149058
6,Santa Rosa,167815
14,Hayward,144186
...,...,...
232,Concord,122067
234,Salinas,150441
236,Los Angeles,3792621
243,Simi Valley,124237


## Problem 3: Compositions of functions
### Part A
Define function `state_to_most_pop_city` which maps each state to its most populous city. Then, compose this function with the function `city_to_population` to get the function `state_to_most_pop_city` $\circ$ `city_to_population`. What does the function you obtain represent?

In [32]:
# this is a bit hacky
state_to_most_pop_idx = cities.groupby("State", as_index=False)[["Population"]].idxmax()[["Population"]].values.flatten()
state_to_most_pop_cities = cities.iloc[state_to_most_pop_idx][["State","City"]]
state_to_most_pop_cities

,State,City
136,Alaska,Anchorage
236,California,Los Angeles
58,Colorado,Denver
36,Hawaii,Honolulu
165,Washington,Seattle
242,Wyoming,Cheyenne


In [33]:
city_to_city1 = state_to_most_pop_cities.merge(city_to_population, on="City", how="left").drop("City", axis=1)
city_to_city1

,State,Population
0,Alaska,291826
1,California,3792621
2,Colorado,600158
3,Hawaii,337256
4,Washington,608660
5,Wyoming,59466


### Part B
Define a _different_ function from `State` to `City` (this could be one of your functions defined in problem 1) and find its composition with the function `city_to_population`.

In [34]:
city_to_city2 = state_to_city2.merge(city_to_population, on="City", how="left").drop("City", axis=1)
city_to_city2

,State,Population
0,Alaska,2369
1,California,106098
2,Colorado,18644
3,Hawaii,11674
4,Washington,91067
5,Wyoming,1807


### Part C
Find a function `f` such that `city_to_population` $\circ$ `f` is the identity function on the set `Population`.  

In [35]:
## there's something slightly weird that happens here because there are a couple city names that appear twice
f = cities.groupby("Population", as_index=False)[["Population", "City"]].first()
ctp_comp_f = f.merge(city_to_population, on = "City", how = "left").drop("City", axis=1)
ctp_comp_f

,Population_x,Population_y
0,212,212
1,681,681
2,688,688
3,721,721
4,755,755
...,...,...
248,608660,608660
249,805235,805235
250,945942,945942
251,1301617,1301617


### Part D
Find a subset $S$ of the set `City` such that when we restrict `city_to_population` to this subset (call this function `city_to_population_S`), `f` $\circ$ `city_to_population_S` is the identity function on the set $S$.

In [43]:
S = cities.groupby("Population", as_index=False).first()[["City"]]
city_to_population_S = city_to_population[city_to_population["City"].isin(S.values.flatten())]
f_comp_ctpS = city_to_population_S.merge(f, on = "Population", how = "left").drop("Population", axis=1)
f_comp_ctpS

,City_x,City_y
0,Corona,Corona
1,Longmont,Longmont
2,Palmdale,Palmdale
3,Star Valley Ranch,Star Valley Ranch
4,Pomona,Pomona
...,...,...
244,Elk Grove,Elk Grove
245,Aiea,Aiea
246,Littleton,Littleton
247,Thornton,Thornton


## Problem 4: Idempotent functions
### Part A
Find an idempotent function `g` on the set `City` which is _not_ the identity function on the set `City`. Show that this function is indeed idempotent by computing `g(City)` and `g(g(City))`.

In [47]:
temp = cities.groupby("State", as_index=False).min()[["State", "City"]]
city_to_state = cities[["City","State"]]
idempotent_fn = city_to_state.merge(temp, on="State", how="left").drop("State", axis=1)
idempotent_fn.columns=["City", "g(City)"]
idempotent_fn

,City,g(City)
0,Corona,Anaheim
1,Longmont,Arvada
2,Palmdale,Anaheim
3,Star Valley Ranch,Afton
4,Pomona,Anaheim
...,...,...
246,Elk Grove,Anaheim
247,Aiea,Ahuimanu
248,Littleton,Arvada
249,Thornton,Arvada


In [52]:
idempotent_fn2 = idempotent_fn.merge(idempotent_fn, left_on="g(City)", right_on="City")[["City_x","g(City)_x","City_y"]]
idempotent_fn2.columns = ["City", "g(City)", "g(g(City))"]
idempotent_fn2

,City,g(City),g(g(City))
0,Corona,Anaheim,Anaheim
1,Palmdale,Anaheim,Anaheim
2,Pomona,Anaheim,Anaheim
3,Santa Rosa,Anaheim,Anaheim
4,Hayward,Anaheim,Anaheim
...,...,...,...
246,Halawa,Ahuimanu,Ahuimanu
247,Maili,Ahuimanu,Ahuimanu
248,Waimalu,Ahuimanu,Ahuimanu
249,Ewa Beach,Ahuimanu,Ahuimanu


### Part B
By restricting the function, show that your function `g` from part A is the identity function on its image.

In [53]:
idempotent_fn[idempotent_fn["City"].isin(idempotent_fn["g(City)"])]

,City,g(City)
33,Auburn,Auburn
37,Afton,Afton
39,Akutan,Akutan
51,Anaheim,Anaheim
125,Ahuimanu,Ahuimanu
133,Arvada,Arvada
